In [2]:
import pandas as pd
import sqlite3

# Load Orders CSV
orders_df = pd.read_csv("orders.csv")
print("Orders Data Loaded")

# Load Users JSON
users_df = pd.read_json("users.json")
print("Users Data Loaded")

# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Read and execute SQL file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)

# Load restaurants table
restaurants_df = pd.read_sql("SELECT * FROM restaurants", conn)
print("Restaurants Data Loaded")

# Merge Orders with Users (LEFT JOIN)
merged_df = pd.merge(
    orders_df,
    users_df,
    on="user_id",
    how="left"
)

# Merge with Restaurants (LEFT JOIN)
final_df = pd.merge(
    merged_df,
    restaurants_df,
    on="restaurant_id",
    how="left"
)

# Save FULL dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("Full Final Dataset Created: final_food_delivery_dataset.csv")
print("Total Rows:", final_df.shape[0])
print("Total Columns:", final_df.shape[1])

Orders Data Loaded
Users Data Loaded
Restaurants Data Loaded
Full Final Dataset Created: final_food_delivery_dataset.csv
Total Rows: 10000
Total Columns: 12


In [3]:
import pandas as pd

# Load final dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Convert date column
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

print("Dataset Loaded")
print("Total Rows:", df.shape[0])
print("-" * 50)

# 1️⃣ City with highest total revenue from Gold members
q1 = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)
print("Q1: City with highest Gold revenue:", q1)

# 2️⃣ Cuisine with highest average order value
q2 = df.groupby("cuisine")["total_amount"].mean().idxmax()
print("Q2: Cuisine with highest average order value:", q2)

# 3️⃣ Distinct users with total order value > 1000
user_totals = df.groupby("user_id")["total_amount"].sum()
count_users = (user_totals > 1000).sum()

if count_users < 500:
    q3 = "< 500"
elif count_users <= 1000:
    q3 = "500 – 1000"
elif count_users <= 2000:
    q3 = "1000 – 2000"
else:
    q3 = "> 2000"

print("Q3: Users with total orders > 1000:", q3)

# 4️⃣ Rating range with highest total revenue
def rating_bucket(r):
    if 3.0 <= r <= 3.5:
        return "3.0 – 3.5"
    elif 3.6 <= r <= 4.0:
        return "3.6 – 4.0"
    elif 4.1 <= r <= 4.5:
        return "4.1 – 4.5"
    elif 4.6 <= r <= 5.0:
        return "4.6 – 5.0"

df["rating_range"] = df["rating"].apply(rating_bucket)

q4 = df.groupby("rating_range")["total_amount"].sum().idxmax()
print("Q4: Rating range with highest revenue:", q4)

# 5️⃣ City with highest average order value among Gold members
q5 = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .idxmax()
)
print("Q5: City with highest Gold AOV:", q5)

# 6️⃣ Cuisine with lowest number of restaurants but high revenue
restaurant_counts = df.groupby("cuisine")["restaurant_id"].nunique()
q6 = restaurant_counts.idxmin()
print("Q6: Cuisine with lowest restaurant count:", q6)

# 7️⃣ Percentage of orders by Gold members
percent_gold = round(
    (df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100
)
print("Q7: Percentage of Gold orders:", f"{percent_gold}%")

# 8️⃣ Restaurant with highest AOV but < 20 orders
restaurant_stats = (
    df.groupby("restaurant_name_x")
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
)

q8 = restaurant_stats[
    restaurant_stats["total_orders"] < 20
]["avg_order_value"].idxmax()

print("Q8: Restaurant with highest AOV (<20 orders):", q8)

# 9️⃣ Combination contributing highest revenue
combo = (
    df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .idxmax()
)

print("Q9: Highest revenue combination:", combo[0], "+", combo[1], "cuisine")

# 🔟 Quarter with highest total revenue
df["quarter"] = df["order_date"].dt.to_period("Q")
q10 = df.groupby("quarter")["total_amount"].sum().idxmax()

print("Q10: Quarter with highest revenue:", q10)

Dataset Loaded
Total Rows: 10000
--------------------------------------------------
Q1: City with highest Gold revenue: Chennai
Q2: Cuisine with highest average order value: Mexican
Q3: Users with total orders > 1000: > 2000
Q4: Rating range with highest revenue: 4.6 – 5.0
Q5: City with highest Gold AOV: Chennai
Q6: Cuisine with lowest restaurant count: Chinese
Q7: Percentage of Gold orders: 50%
Q8: Restaurant with highest AOV (<20 orders): Hotel Dhaba Multicuisine
Q9: Highest revenue combination: Regular + Mexican cuisine
Q10: Quarter with highest revenue: 2023Q3


In [4]:
import pandas as pd
import sqlite3

# Load datasets
orders_df = pd.read_csv("orders.csv")
users_df = pd.read_json("users.json")

# Load restaurants from SQL
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as file:
    cursor.executescript(file.read())

restaurants_df = pd.read_sql("SELECT * FROM restaurants", conn)

# Merge datasets
final_df = (
    orders_df
    .merge(users_df, on="user_id", how="left")
    .merge(restaurants_df, on="restaurant_id", how="left")
)

# Column used to join orders.csv and users.json
print("Join column (orders & users):", "user_id")

# Dataset format containing cuisine and rating
print("Cuisine & rating stored in:", "SQL format")

# Total rows in final merged dataset
print("Total rows in final dataset:", final_df.shape[0])

# Values when no matching user record exists
print("Missing user values appear as:", final_df.isna().any().any())

# Pandas function used to combine datasets
print("Pandas function used for join:", "merge()")

# Source of membership column
print("Membership column source:", "users.json")

# Join key for orders and restaurants
print("Join key for restaurant data:", "restaurant_id")

# Column identifying food type
print("Food type column:", "cuisine")

# User details repetition check
sample_user = final_df["user_id"].value_counts().idxmax()
print(
    "User details repeat for multiple orders:",
    final_df[final_df["user_id"] == sample_user].shape[0] > 1
)

Join column (orders & users): user_id
Cuisine & rating stored in: SQL format
Total rows in final dataset: 10000
Missing user values appear as: False
Pandas function used for join: merge()
Membership column source: users.json
Join key for restaurant data: restaurant_id
Food type column: cuisine
User details repeat for multiple orders: True
